<a href="https://colab.research.google.com/github/brijithroshan/Analytics/blob/main/Hackathon1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

summer_analytics_mid_hackathon_path = kagglehub.competition_download('summer-analytics-mid-hackathon')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv")
df #ignore the warnings

In [ ]:
df.isnull().sum()

In [ ]:
df.fillna(df.mean(numeric_only=True), inplace=True) #simple mean imputation [This part has a lot of scope for imporovement.]
#keep in mind that the data is inherently noisy and the test dataset is not.
df.isnull().sum()

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column (if it's categorical)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split into features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Fit multinomial logistic regression
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=10
)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Classification report with all original class labels
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


In [ ]:
test_data = pd.read_csv("/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv")
test_data.shape

In [ ]:
ID=test_data['ID']
test_data.drop(['ID'],axis=1,inplace=True)

In [ ]:
y_test = model.predict(test_data)

In [ ]:
y_test

In [ ]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

In [ ]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [ ]:
result

In [ ]:
result.to_csv("submission.csv", index=False) #this file will appear under the output section of the right navbar. You need to submit this csv file

In [ ]:
# Imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

# Load Data
train = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')

# new edit
X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, stratify=y_train, random_state=42)

model.fit(X_tr, y_tr)
val_preds = model.predict(X_val)

# 🧾 Print classification report
print(classification_report(y_val, val_preds, target_names=le.classes_))

# Handle missing values using mean imputation
train.fillna(train.mean(numeric_only=True), inplace=True)
test.fillna(test.mean(numeric_only=True), inplace=True)

# Feature Engineering
def extract_features(df):
    ndvi_cols = [col for col in df.columns if '_N' in col]
    features = pd.DataFrame(index=df.index)
    features['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
    features['ndvi_std'] = df[ndvi_cols].std(axis=1)
    features['ndvi_min'] = df[ndvi_cols].min(axis=1)
    features['ndvi_max'] = df[ndvi_cols].max(axis=1)
    features['ndvi_median'] = df[ndvi_cols].median(axis=1)
    features['ndvi_range'] = features['ndvi_max'] - features['ndvi_min']
    features['ndvi_first'] = df[ndvi_cols].iloc[:, 0]
    features['ndvi_last'] = df[ndvi_cols].iloc[:, -1]
    features['ndvi_trend'] = features['ndvi_last'] - features['ndvi_first']
    features['ndvi_slope'] = df[ndvi_cols].apply(lambda row: np.polyfit(range(len(row)), row, 1)[0], axis=1)
    return features


X_train = extract_features(train)
X_test = extract_features(test)

# Encode target labels
le = LabelEncoder()
y_train = le.fit_transform(train['class'])

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, C=1.0, class_weight= 'balanced')
model.fit(X_train_scaled, y_train)

# Evaluate on hold-out split
X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, stratify=y_train, random_state=42)
model.fit(X_tr, y_tr)
val_preds = model.predict(X_val)
print(classification_report(y_val, val_preds, target_names=le.classes_))

# Predict on test set
test_preds = model.predict(X_test_scaled)
test_labels = le.inverse_transform(test_preds)

# Generate submission
submission = pd.DataFrame({
    'ID': test['ID'],
    'class': test_labels
})
submission.to_csv('submission.csv', index=False)
submission.head()
